<a href="https://colab.research.google.com/github/hungpham13/Laptop-EDA/blob/main/DS_Project_20211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[Guideline](https://husteduvn.sharepoint.com/sites/IntroductiontoDataScience-IT4142E-2021-2022/Class%20Materials/Capstone%20project/DS-Capstone%20project%20guidelines.pdf)**
## Topic description assignment (20th of October, 2021)
[Our project proposal](https://docs.google.com/document/d/16L1Mqy_HtDfhxB0xNXmCZSBNMKazWsYgwA9s1Vueih8/edit?usp=sharing)


## Assginment for each member
### **[On GG Sheet](https://docs.google.com/spreadsheets/d/11ldCYnPJwYuMKBba6jX7qPYnYtekAYjR9YvORww76hE/edit#)**

Distinguish between programming tasks and analytic tasks. 
- Examples of programming tasks: data scraping for source A; implementing algorithm A,
etc.
- Examples of analytic tasks: proposing our subject, selecting the algorithms to be used to
solve this problem, writing parts 1/2/3/4 of the report, writing parts 1/2/3/4 of the presentation, etc.

For each task, provide a detailed contribution of each member to this task:
- e.g.: Programming Task 1- data scraping from source A: member1 Mr X 40% ; member 2 Mrs Y 60% (no more than 2 members /task for programming tasks)...).
- e.g.: Analytic Task 1– proposing our subject: member1 Mr X 40% ; member 2 Mrs Y 30% ; member 2 Mr Z 30% (can be more than 2 members /task for analytic
tasks)...)

## Final assignment (29th of December, 2021)
    
Create 1 directory named with group number, upload on the dedicated assignment on Teams:
- Report: PDF file, no more than 15 pages (not included reference pages / first page / table of contents)
- Code: put in a single .zip file, with a readme.txt file explaining your codes structure, how to run the project – if necessary.
- Video demo of your Python code running in a notebook
- Presentation: PDF file, no more than 10 slides. The first slide MUST contain the (recent) pictures of the students in the group, with student’s name/number

# Materials

1.Scrapy, Selenium, BeautifulSoup –Data scraping
2.Numpy, Pandas, Seaborn –data analysis and visualization

[Kaggle detailed EDA](https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python)

[Github DS project](https://github.com/veb-101/Data-Science-Projects)

[Havard Data Science Course](http://cs109.github.io/2015/pages/videos.html)

[Web Scraping with Python (Book)](https://vn1lib.org/book/3515980/5d50aa)

[Web Scraping on Kaggle](https://www.kaggle.com/getting-started?search=web+scraping)